In [1]:
import os
import numpy as np
from six.moves import cPickle
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
from keras import backend as K
from keras.models import Model, model_from_json
from keras.layers import Input, Dense, Flatten

from prednet import PredNet
from data_utils import SequenceGenerator

from tqdm import tqdm

Using TensorFlow backend.


In [2]:
n_plot = 40
batch_size = 10
nt = 24

In [3]:
WEIGHTS_DIR = '../Training/weights/'
DATA_DIR = '../data/'
RESULTS_SAVE_DIR = './weather_results/'

In [4]:
weights_file = os.path.join(WEIGHTS_DIR, 'prednet_weather_weights.hdf5')
json_file = os.path.join(WEIGHTS_DIR, 'prednet_weather_model.json')
test_file = os.path.join(DATA_DIR, 'x_test.hkl')
test_sources = os.path.join(DATA_DIR, 'sources_test.hkl')

In [5]:
# Load trained model
f = open(json_file, 'r')
json_string = f.read()
f.close()
train_model = model_from_json(json_string, custom_objects = {'PredNet': PredNet})
train_model.load_weights(weights_file)

In [6]:
# Create testing model (to output predictions)
layer_config = train_model.layers[1].get_config()
layer_config['output_mode'] = 'prediction'
data_format = layer_config['data_format'] if 'data_format' in layer_config else layer_config['dim_ordering']
test_prednet = PredNet(weights=train_model.layers[1].get_weights(), **layer_config)
input_shape = list(train_model.layers[0].batch_input_shape[1:])
input_shape[0] = nt
inputs = Input(shape=tuple(input_shape))
predictions = test_prednet(inputs)
test_model = Model(inputs=inputs, outputs=predictions)

In [7]:
test_generator = SequenceGenerator(test_file, test_sources, nt, sequence_start_mode='unique', data_format=data_format)
X_test = test_generator.create_all()
X_hat = test_model.predict(X_test, batch_size)
if data_format == 'channels_first':
    X_test = np.transpose(X_test, (0, 1, 3, 4, 2))
    X_hat = np.transpose(X_hat, (0, 1, 3, 4, 2))

In [8]:
# Compare MSE of PredNet predictions vs. using last frame.  Write results to prediction_scores.txt
mse_model = np.nanmean( (X_test[:, 1:] - X_hat[:, 1:])**2 )  # look at all timesteps except the first
mse_prev = np.nanmean( (X_test[:, :-1] - X_test[:, 1:])**2 )
if not os.path.exists(RESULTS_SAVE_DIR): os.mkdir(RESULTS_SAVE_DIR)
f = open(RESULTS_SAVE_DIR + 'prediction_scores.txt', 'w')
f.write("Model MSE: %f\n" % mse_model)
f.write("Previous Frame MSE: %f" % mse_prev)
f.close()

In [9]:
print("Model MSE:\t {}\nPrev Frame MSE:\t {}".format(mse_model,mse_prev))

Model MSE:	 14.119876861572266
Prev Frame MSE:	 0.02834348939359188


In [ ]:
# Plot some predictions
aspect_ratio = float(X_hat.shape[3]) / X_hat.shape[2]
plt.figure(figsize = (nt, 7*2*aspect_ratio))
gs = gridspec.GridSpec(2*7, nt)
gs.update(wspace=0., hspace=0.2)
plot_save_dir = os.path.join(RESULTS_SAVE_DIR, 'prediction_plots/')
if not os.path.exists(plot_save_dir): os.mkdir(plot_save_dir)
plot_idx = np.random.permutation(X_test.shape[0])[:n_plot]
for i in tqdm(plot_idx):
    for t in range(nt):
        for c in range(7):
            plt.subplot(gs[t + c*2*nt])
            plt.imshow(X_test[i,t,:,:,c], interpolation='none')
            plt.tick_params(axis='both', which='both', bottom='off', top='off', left='off', right='off', labelbottom='off', labelleft='off')
            if t==0: plt.ylabel('Actual', fontsize=10)

            plt.subplot(gs[t + (c*2+1)*nt])
            plt.imshow(X_hat[i,t,:,:,c], interpolation='none')
            plt.tick_params(axis='both', which='both', bottom='off', top='off', left='off', right='off', labelbottom='off', labelleft='off')
            if t==0: plt.ylabel('Predicted', fontsize=10)

    plt.savefig(plot_save_dir +  'plot_' + str(i) + '.png')
    plt.clf()

 63%|███████████████████████████████████████████████████▉                              | 19/30 [09:38<05:35, 30.47s/it]

In [ ]:
fig=plt.figure(figsize=(15,10))
columns = 3
rows = 4
for i in range(1,columns+rows +1):
    fig.add_subplot(rows,columns,i)
    plt.imshow(X_test[0,0,:,:,i-1],X_hat[0,0,:,:,i-1])

In [ ]:
X_hat[0][0][0][0][2]